In [ ]:
from random import choice
from copy import copy
from typing import Tuple, Callable

In [ ]:
def successors_by_predecessors(predecessors: list[list[int]]):
    size = len(predecessors)
    return [[succ for succ in range(size) if i in predecessors[succ]] for i in range(size)]

def calculate_critical_times(
        duration: list[int],
        predecessors: list[list[int]],
        successors: list[list[int]] = None) -> Tuple[list[int], list[int]]:
    """Расчёт ранних времён начала и поздних времён завершения работ

    Args:
        duration (list[int]): _description_
        predecessors (list[list[int]]): списки предшествующих работ
        successors (list[list[int]]): списки последующих работ

    Raises:
        ValueError:

    Returns:
        Tuple[list[int], list[int]]: ранние времена начала и поздние времена завершения
    """
    if not successors:
        successors = successors_by_predecessors(predecessors)
    if len(duration) != len(predecessors) or len(predecessors) != len(successors):
        raise ValueError("Invalid data")
    earliest_start = [0 for _ in duration]
    latest_finish = [0 for _ in duration]

    def _calc_earliest_start(index):
        if index:
            earliest_start[index] = max(_calc_earliest_start(pred) + duration[pred] for pred in predecessors[index])
        return earliest_start[index]

    def _calc_latest_finish(index):
        if index == len(successors) - 1:
            latest_finish[index] = earliest_start[index]
        else:
            latest_finish[index] = min(_calc_latest_finish(succ) - duration[succ] for succ in successors[index])
        return latest_finish[index]

    _calc_earliest_start(len(predecessors) - 1)
    _calc_latest_finish(0)

    return earliest_start, latest_finish

class TimeCapacityNode:
    """
    Вспомогательная структура связного списка моментов времени и
    имеющихся в них остаточных запасов ресурсов
    """
    def __init__(self, time: int, capacity: list[int]):
        self.time = time
        self.capacity = capacity
        self.next = None
        self.prev = None

    def insert_after(self, time: int) -> 'TimeCapacityNode':
        if time <= self.time:
            raise ValueError("Invalid time")

        new_node = self.__class__(time, copy(self.capacity))
        new_node.prev = self
        new_node.next = self.next
        if self.next:
            self.next.prev = new_node
        self.next = new_node
        return new_node

    def find_first(self, time: int) -> 'TimeCapacityNode':
        node = self
        while node.time < time:
            if node.next:
                node = node.next
            else:
                return node
        return node.prev

    def enough_resources(self, demand: list[int]) -> bool:
        return all(self.capacity[i] >= demand[i] for i in range(len(self.capacity)))

    def consume(self, demand: list[int]) -> None:
        for i in range(len(self.capacity)):
            self.capacity[i] -= demand[i]

class ActivityListDecoder:
    def decode(
            self,
            activity_list: list[int],
            duration: list[int],
            predecessors: list[list[int]],
            renewable_demands: list[list[int]],
            renewable_capacity: list[int]) -> list[int]:
        """
        Последовательная схема генерации расписания для декодирования Activity List.

        Args:
            activity_list (list[int]): закодированное решение (Activity List)
            duration (list[int]): продолжительности работ
            predecessors (list[list[int]]): списки предшествующих работ
            renewable_demands (list[list[int]]): затраты неисчерпаемых ресурсов
            renewable_capacity (list[int]): запасы неисчерпаемых ресурсов

        Raises:
            ValueError: выбрасывается при нарушении связей предшествования

        Returns:
            list[int]: времена начала работ
        """
        count = len(activity_list)
        root_node = TimeCapacityNode(0, copy(renewable_capacity)) # Связный список моментов изменения запаса ресурсов
        starts = [0] * count
        finish_nodes = [None] * count
        finish_nodes[0] = root_node

        for i in activity_list:
            # Работа может начаться не раньше, чем её последняя предшественница
            start_node = root_node
            for pred in predecessors[i]:
                if not finish_nodes[pred]:
                    raise ValueError("Invalid activity list")
                if finish_nodes[pred].time > start_node.time:
                    start_node = finish_nodes[pred]
            # Ищем такую позицию для начала, при которой не нарушатся ресурсные ограничения
            start_node, last_node, finish_node, finish_time = self._find_position(
                start_node, duration[i], renewable_demands[i]
            )
            starts[i] = start_node.time
            if not finish_node or finish_node.time != finish_time:
                finish_node = last_node.insert_after(finish_time)
            finish_nodes[i] = finish_node
            # Обновляем доступное число ресурсов в моменты времени, затрагиваемые данной работой
            self._consume(start_node, finish_node, renewable_demands[i])

        return starts

    def _consume(
            self, start_node: TimeCapacityNode,
            finish_node: TimeCapacityNode,
            demand: list[int]) -> None:
        node = start_node
        while node != finish_node:
            node.consume(demand)
            node = node.next

    def _find_position(
            self, start_node: int, duration: int, demand: list[int]
            ) -> Tuple[TimeCapacityNode, TimeCapacityNode, TimeCapacityNode, int]:
        if not duration:
            return (start_node, start_node, start_node, start_node.time)

        finish_time = start_node.time + duration
        t = start_node.find_first(finish_time)
        last_node = t
        t_test = start_node

        while t != t_test.prev:
            if t.enough_resources(demand):
                t = t.prev
            else:
                start_node = t.next
                finish_time = start_node.time + duration
                if last_node.next:
                    t_test = last_node.next
                    last_node = t_test.find_first(finish_time)
                    t = last_node
                else:
                    break

        return (start_node, last_node, last_node.next, finish_time)

class ActivityListSampler:
    def __init__(
            self,
            predecessors: list[list[int]],
            successors: list[list[int]] = None) -> None:
        """
        Args:
            predecessors (list[list[int]]): списки предшествующих работ
            successors (list[list[int]]): списки последующих работ
        """
        self.predecessors = predecessors
        self.size = len(predecessors)
        if not successors:
            successors = successors_by_predecessors(predecessors)
        self.successors = successors

    def _generate(self, func: Callable = None) -> list[int]:
        result = []
        ramain_predecessors = [set(pred) for pred in self.predecessors]
        ready_set = [i for i in range(self.size) if not self.predecessors[i]]

        for _ in range(self.size):
            if not ready_set:
                raise ValueError("Incorrect project network")

            next_activity = func(ready_set) if func else choice(ready_set)
            ready_set.remove(next_activity)
            result.append(next_activity)

            for successor in self.successors[next_activity]:
                ramain_predecessors[successor].remove(next_activity)
                if not ramain_predecessors[successor]:
                    ready_set.append(successor)

        return result

    def generate_random(self) -> list[int]:
        """
        Генерация Activity List случайным образом

        Returns:
            list[int]: Activity List
        """
        return self._generate()

    def generate_by_max_rule(self, rule: Callable) -> list[int]:
        """
        Генерация Activity List упорядочивая по убыванию метрики

        Returns:
            list[int]: Activity List
        """
        def func(data: list[int]):
            return max(data, key=rule)
        return self._generate(func)

    def generate_by_min_rule(self, rule: Callable) -> list[int]:
        """
        Генерация Activity List упорядочивая по возрастанию метрики

        Returns:
            list[int]: Activity List
        """
        def func(data: list[int]):
            return min(data, key=rule)
        return self._generate(func)

Пример на диске

In [ ]:
predecessors = [[], [0], [1], [1], [1], [4], [2,3,5], [6]]
# Длительности
durations = [0, 5, 15, 5, 8, 2, 8, 0]

In [ ]:
# Рассчёт ранних времён начала и поздних времён окончания
earliest_start, latest_finish = calculate_critical_times(durations, predecessors)

In [ ]:
earliest_start

In [ ]:
latest_finish

In [ ]:
# Ресурсные затраты
renewable_demands = [[0, 0], [2, 2], [2, 3], [3, 1], [4, 0], [2, 2], [0, 4], [0, 0]]
renewable_capacities = [5, 6]

In [ ]:
sampler = ActivityListSampler(predecessors)
decoder = ActivityListDecoder()

In [ ]:
# Генерируем случайный порядок
random_activity_list = sampler.generate_random()
start_times = decoder.decode(random_activity_list, durations, predecessors, renewable_demands, renewable_capacities)

In [ ]:
random_activity_list

In [ ]:
start_times

In [ ]:
# Генерируем порядок в соответствии с правилом (по возрастанию поздних времен конца)
heuristic_activity_list = sampler.generate_by_min_rule(lambda j: latest_finish[j])
start_times2 = decoder.decode(random_activity_list, durations, predecessors, renewable_demands, renewable_capacities)

In [ ]:
heuristic_activity_list

In [ ]:
start_times2

Наш проект — **SmartMenu AI**: мобильное приложение «Готовь из того, что есть» (Computer Vision + генеративный ИИ).

Ниже приведён расчёт календарного плана (RCPSP) по эвристикам и выбор лучшей эвристики на основе минимизации makespan.

In [ ]:
predecessors = [
    [],  # 0 — Старт (фиктивная),
    [0],  # 1 — Сбор требований и согласование Use Cases,
    [1],  # 2 — UX/UI: прототипы экранов + дизайн-система,
    [1],  # 3 — Бэкенд: пользователи/профиль/настройки,
    [1],  # 4 — Каталог продуктов (офлайн-база) и справочники,
    [2, 4],  # 5 — CV: сканирование холодильника (интеграция модели),
    [2, 4],  # 6 — OCR: сканер чеков (пайплайн распознавания),
    [3, 4],  # 7 — Inventory: виртуальная кладовка + сроки годности,
    [3],  # 8 — Предпочтения и фильтры питания (кето/веган/…),
    [7, 8],  # 9 — AI Core: генерация рецепта (LLM-сервис),
    [7, 9],  # 10 — Мэтчинг рецептов 100%/80% (логика подбора),
    [2, 10],  # 11 — UI рецепта: пошаговые инструкции + карточка блюда,
    [10],  # 12 — Список докупки и подсказки «чего не хватает»,
    [7],  # 13 — Уведомления по срокам годности,
    [3, 5, 6, 7, 9],  # 14 — Аналитика/логирование/метрики качества распознавания,
    [1],  # 15 — QA: тест-план и тест-кейсы (unit/integration/e2e),
    [7, 9, 10],  # 16 — API-шлюз/контракты: объединение сервисов,
    [2, 5],  # 17 — Мобильный клиент: камера + разметка распознавания,
    [2, 7],  # 18 — Мобильный клиент: экраны кладовки/редактирование,
    [2, 10, 11],  # 19 — Мобильный клиент: экраны рецептов/фильтры/поиск,
    [12, 13, 16, 17, 18, 19, 15],  # 20 — Интеграция и E2E: скан → рецепт → списание → уведомления,
    [3],  # 21 — CI/CD и окружения (dev/test/prod), сборки,
    [20, 14],  # 22 — Документация и финальная упаковка результата,
    [21, 22],  # 23 — Финиш (фиктивная)
]


In [ ]:
durations = [
    0,  # 0 — Старт (фиктивная),
    3,  # 1 — Сбор требований и согласование Use Cases,
    5,  # 2 — UX/UI: прототипы экранов + дизайн-система,
    4,  # 3 — Бэкенд: пользователи/профиль/настройки,
    3,  # 4 — Каталог продуктов (офлайн-база) и справочники,
    6,  # 5 — CV: сканирование холодильника (интеграция модели),
    5,  # 6 — OCR: сканер чеков (пайплайн распознавания),
    6,  # 7 — Inventory: виртуальная кладовка + сроки годности,
    3,  # 8 — Предпочтения и фильтры питания (кето/веган/…),
    6,  # 9 — AI Core: генерация рецепта (LLM-сервис),
    4,  # 10 — Мэтчинг рецептов 100%/80% (логика подбора),
    4,  # 11 — UI рецепта: пошаговые инструкции + карточка блюда,
    3,  # 12 — Список докупки и подсказки «чего не хватает»,
    3,  # 13 — Уведомления по срокам годности,
    4,  # 14 — Аналитика/логирование/метрики качества распознавания,
    3,  # 15 — QA: тест-план и тест-кейсы (unit/integration/e2e),
    3,  # 16 — API-шлюз/контракты: объединение сервисов,
    5,  # 17 — Мобильный клиент: камера + разметка распознавания,
    5,  # 18 — Мобильный клиент: экраны кладовки/редактирование,
    5,  # 19 — Мобильный клиент: экраны рецептов/фильтры/поиск,
    6,  # 20 — Интеграция и E2E: скан → рецепт → списание → уведомления,
    3,  # 21 — CI/CD и окружения (dev/test/prod), сборки,
    4,  # 22 — Документация и финальная упаковка результата,
    0,  # 23 — Финиш (фиктивная)
]


### Справочник работ (индекс → описание)

- **0**: 0 — Старт (фиктивная)
- **1**: 1 — Сбор требований и согласование Use Cases
- **2**: 2 — UX/UI: прототипы экранов + дизайн-система
- **3**: 3 — Бэкенд: пользователи/профиль/настройки
- **4**: 4 — Каталог продуктов (офлайн-база) и справочники
- **5**: 5 — CV: сканирование холодильника (интеграция модели)
- **6**: 6 — OCR: сканер чеков (пайплайн распознавания)
- **7**: 7 — Inventory: виртуальная кладовка + сроки годности
- **8**: 8 — Предпочтения и фильтры питания (кето/веган/…)
- **9**: 9 — AI Core: генерация рецепта (LLM-сервис)
- **10**: 10 — Мэтчинг рецептов 100%/80% (логика подбора)
- **11**: 11 — UI рецепта: пошаговые инструкции + карточка блюда
- **12**: 12 — Список докупки и подсказки «чего не хватает»
- **13**: 13 — Уведомления по срокам годности
- **14**: 14 — Аналитика/логирование/метрики качества распознавания
- **15**: 15 — QA: тест-план и тест-кейсы (unit/integration/e2e)
- **16**: 16 — API-шлюз/контракты: объединение сервисов
- **17**: 17 — Мобильный клиент: камера + разметка распознавания
- **18**: 18 — Мобильный клиент: экраны кладовки/редактирование
- **19**: 19 — Мобильный клиент: экраны рецептов/фильтры/поиск
- **20**: 20 — Интеграция и E2E: скан → рецепт → списание → уведомления
- **21**: 21 — CI/CD и окружения (dev/test/prod), сборки
- **22**: 22 — Документация и финальная упаковка результата
- **23**: 23 — Финиш (фиктивная)

In [ ]:
len(predecessors)

In [ ]:
len(durations)

In [ ]:
# Рассчёт ранних времён начала и поздних времён окончания
earliest_start, latest_finish = calculate_critical_times(durations, predecessors)

In [ ]:
earliest_start

In [ ]:
latest_finish

In [ ]:
renewable_demands = [
    [0, 0, 0, 0, 0, 0],  # 0 — Старт (фиктивная),
    [1, 1, 0, 0, 0, 0],  # 1 — Сбор требований и согласование Use Cases,
    [1, 0, 0, 1, 0, 1],  # 2 — UX/UI: прототипы экранов + дизайн-система,
    [1, 0, 1, 0, 1, 0],  # 3 — Бэкенд: пользователи/профиль/настройки,
    [0, 1, 0, 0, 1, 0],  # 4 — Каталог продуктов (офлайн-база) и справочники,
    [0, 1, 1, 0, 1, 1],  # 5 — CV: сканирование холодильника (интеграция модели),
    [0, 1, 1, 0, 1, 0],  # 6 — OCR: сканер чеков (пайплайн распознавания),
    [1, 1, 1, 0, 1, 1],  # 7 — Inventory: виртуальная кладовка + сроки годности,
    [0, 1, 0, 0, 1, 0],  # 8 — Предпочтения и фильтры питания (кето/веган/…),
    [0, 1, 1, 0, 1, 0],  # 9 — AI Core: генерация рецепта (LLM-сервис),
    [0, 1, 1, 0, 1, 0],  # 10 — Мэтчинг рецептов 100%/80% (логика подбора),
    [0, 0, 1, 1, 0, 1],  # 11 — UI рецепта: пошаговые инструкции + карточка блюда,
    [0, 1, 1, 0, 1, 1],  # 12 — Список докупки и подсказки «чего не хватает»,
    [0, 0, 1, 0, 1, 1],  # 13 — Уведомления по срокам годности,
    [0, 1, 0, 0, 1, 0],  # 14 — Аналитика/логирование/метрики качества распознавания,
    [1, 0, 1, 0, 0, 0],  # 15 — QA: тест-план и тест-кейсы (unit/integration/e2e),
    [1, 1, 0, 0, 1, 0],  # 16 — API-шлюз/контракты: объединение сервисов,
    [0, 0, 1, 0, 0, 1],  # 17 — Мобильный клиент: камера + разметка распознавания,
    [0, 0, 1, 1, 0, 1],  # 18 — Мобильный клиент: экраны кладовки/редактирование,
    [0, 0, 1, 1, 0, 1],  # 19 — Мобильный клиент: экраны рецептов/фильтры/поиск,
    [1, 1, 1, 0, 1, 1],  # 20 — Интеграция и E2E: скан → рецепт → списание → уведомления,
    [1, 0, 0, 0, 1, 0],  # 21 — CI/CD и окружения (dev/test/prod), сборки,
    [1, 0, 0, 0, 0, 0],  # 22 — Документация и финальная упаковка результата,
    [0, 0, 0, 0, 0, 0],  # 23 — Финиш (фиктивная)
]


In [ ]:
len(renewable_demands)

In [ ]:
renewable_capacities = [1, 1, 1, 1, 2, 2]


In [ ]:
sampler = ActivityListSampler(predecessors)
decoder = ActivityListDecoder()

In [ ]:
# Генерируем случайный порядок
random_activity_list = sampler.generate_random()
start_times = decoder.decode(random_activity_list, durations, predecessors, renewable_demands, renewable_capacities)

In [ ]:
random_activity_list

In [ ]:
start_times

In [ ]:
# Генерируем порядок в соответствии с правилом (по возрастанию поздних времен конца)
heuristic_activity_list = sampler.generate_by_min_rule(lambda j: latest_finish[j])
start_times2 = decoder.decode(random_activity_list, durations, predecessors, renewable_demands, renewable_capacities)

In [ ]:
heuristic_activity_list

In [ ]:
start_times2

In [ ]:
# Генерируем порядок в соответствии с правилом (по возрастанию поздних времен конца)
heuristic_activity_list3 = sampler.generate_by_max_rule(lambda j: latest_finish[j])
start_times3 = decoder.decode(random_activity_list, durations, predecessors, renewable_demands, renewable_capacities)

In [ ]:
heuristic_activity_list3

In [ ]:
start_times3

Дополнение своими эвристиками

In [ ]:
from random import expovariate
import numpy as np

In [ ]:
class ActivityListSampler:
    def __init__(
            self,
            predecessors: list[list[int]],
            successors: list[list[int]] = None) -> None:
        """
        Args:
            predecessors (list[list[int]]): списки предшествующих работ
            successors (list[list[int]]): списки последующих работ
        """
        self.predecessors = predecessors
        self.size = len(predecessors)
        if not successors:
            successors = successors_by_predecessors(predecessors)
        self.successors = successors

    def _generate(self, func: Callable = None) -> list[int]:
        result = []
        ramain_predecessors = [set(pred) for pred in self.predecessors]
        ready_set = [i for i in range(self.size) if not self.predecessors[i]]

        for _ in range(self.size):
            if not ready_set:
                raise ValueError("Incorrect project network")

            next_activity = func(ready_set) if func else choice(ready_set)
            ready_set.remove(next_activity)
            result.append(next_activity)

            for successor in self.successors[next_activity]:
                ramain_predecessors[successor].remove(next_activity)
                if not ramain_predecessors[successor]:
                    ready_set.append(successor)

        return result

    def generate_random(self) -> list[int]:
        """
        Генерация Activity List случайным образом

        Returns:
            list[int]: Activity List
        """
        return self._generate()

    def generate_by_max_rule(self, rule: Callable) -> list[int]:
        """
        Генерация Activity List упорядочивая по убыванию метрики

        Returns:
            list[int]: Activity List
        """
        def func(data: list[int]):
            return max(data, key=rule)
        return self._generate(func)

    def generate_by_min_rule(self, rule: Callable) -> list[int]:
        """
        Генерация Activity List упорядочивая по возрастанию метрики

        Returns:
            list[int]: Activity List
        """
        def func(data: list[int]):
            return min(data, key=rule)
        return self._generate(func)

    # SLK - по возрастанию общего резерва - значит мин
    def generate_by_SLK(self, latest_finish: list[int], earliest_start: list[int], duration: list[int]) -> list[int]:
        def slack(j):
            return latest_finish[j] - earliest_start[j] - duration[j]
        return self.generate_by_min_rule(slack)

    # FREE - по возрастанию свободного резерва
    # FREE(j) = min(EST(k)) - (EST(j) + duration[j])
    # FREE(j) = min(EST(k)) - EST(j) - duration[j]
    def generate_by_FREE(self, earliest_start: list[int], duration: list[int], successors: list[list[int]]) -> list[int]:
        def free_slack(j):
            if not successors[j]:
                return 0
            return min(earliest_start[succ] for succ in self.successors[j]) - earliest_start[j] - duration[j]
        return self.generate_by_min_rule(free_slack)

    # LST - по возрастанию позднего времени начала
    def generate_by_LST(self, latest_finish: list[int], duration: list[int]) -> list[int]:
        def latest_start(j):
            return latest_finish[j] - duration[j]
        return self.generate_by_min_rule(latest_start)

    # LFT - по возрастанию позднего времени завершения
    def generate_by_LFT(self, latest_finish: list[int]) -> list[int]:
        return self.generate_by_min_rule(lambda j: latest_finish[j])

    def generate_by_exp(self, rule: Callable) -> list[int]:

        def func(data: list[int]):
            return np.random.exponential(scale=1.0, size=None)
        return self._generate(func)

    def generate_by_exp(self, rule: Callable, lambda_param: float = 1.0) -> list[int]:
        """
        Версия с экспоненциальным распределением
        """
        def func(data: list[int]):
            # Генерируем экспоненциально распределенные веса
            weights = [expovariate(lambda_param) for _ in range(len(data))]

            # Выбираем задачу с наименьшим весом (экспоненциальное распределение)
            return data[np.argmin(weights)]

        return self._generate(func)

In [ ]:
sampler = ActivityListSampler(predecessors)
decoder = ActivityListDecoder()

In [ ]:
# Применение всех эвристик
solutions = {}

# Доп. эвристики
solutions['SLK'] = sampler.generate_by_SLK(latest_finish, earliest_start, durations)
solutions['FREE'] = sampler.generate_by_FREE(latest_finish, earliest_start, durations)
solutions['LST'] = sampler.generate_by_LST(latest_finish, durations)
solutions['LFT'] = sampler.generate_by_LFT(latest_finish)
solutions['EXP'] = sampler.generate_by_exp(lambda j: latest_finish[j])

In [ ]:
# Поиск лучшего решения
best_duration = float('inf')
best_solution = None
best_name = ""

for name, activity_list in solutions.items():
    try:
        start_times = decoder.decode(activity_list, durations, predecessors, renewable_demands, renewable_capacities)
        print(name)
        print(start_times)
        makespan = max(start_times[i] + durations[i] for i in range(len(durations)))

        if makespan < best_duration:
            best_duration = makespan
            best_solution = activity_list
            best_name = name
    except:
        continue

print(f"Лучшая эвристика: {best_name} с длительностью {best_duration}")

Перерасчет лучшего решения на рабочее время

In [ ]:
import datetime
from datetime import timedelta

In [ ]:
def calculate_calendar_schedule_simple(start_times: list[int], durations: list[int], start_date: datetime.date):
    """Упрощенный расчет календарного графика"""

    def work_days_to_calendar(work_days: int) -> datetime.date:
        """Конвертирует рабочие дни в календарную дату"""
        calendar_days = work_days
        full_weeks = work_days // 5
        remaining_days = work_days % 5
        total_calendar_days = work_days + full_weeks * 2

        date = start_date + timedelta(days=total_calendar_days)

        # Корректировка если дата попадает на выходные
        while date.weekday() >= 5:  # 5=суббота, 6=воскресенье
            date += timedelta(days=1)
            total_calendar_days += 1

        return date, total_calendar_days

    schedule = []
    for i, work_start in enumerate(start_times):
        start_date_cal, start_cal_days = work_days_to_calendar(work_start)

        # Расчет окончания с учетом длительности
        work_finish = work_start + durations[i]
        finish_date_cal, finish_cal_days = work_days_to_calendar(work_finish)

        schedule.append({
            'task': i,
            'work_start': work_start,
            'work_finish': work_finish,
            'calendar_start': start_date_cal,
            'calendar_finish': finish_date_cal,
            'duration': durations[i],
            'calendar_days': finish_cal_days - start_cal_days
        })

    return schedule

# Расчет календарного графика
calendar_schedule = calculate_calendar_schedule_simple(start_times, durations, datetime.date(2024, 2, 1))

In [ ]:
# Определение ключевых этапов на основе реальных задач
milestones = {
    'Начало проекта': 0,
    'Завершение аутентификации': 5,  # задача 5 (механизм аутентификации)
    'Готовность основных финансовых операций': 25,  # задачи 6,7,8 (Мои финансы + доход/расход)
    'Завершение системы уведомлений': 36,  # задача 14 (пуш-уведомления)
    'Готовность аналитики': 57,  # задача 17 (раздел Аналитика)
    'Завершение проекта': 66  # конечная задача
}

print("ПЛАНОВЫЙ ГРАФИК ПРОЕКТА")
print("=" * 60)
print(f"Начало проекта: {start_date.strftime('%d.%m.%Y')}")

# Расчет календарного графика для ключевых этапов
milestone_schedule = []
for milestone_name, work_day in milestones.items():
    # Для каждой вехи получаем календарную дату
    schedule_item = calculate_calendar_schedule_simple([work_day], [0], start_date)[0]
    milestone_schedule.append({
        'name': milestone_name,
        'work_day': work_day,
        'date': schedule_item['calendar_start']
    })

print("\nКЛЮЧЕВЫЕ ЭТАПЫ:")
print("-" * 40)
for milestone in milestone_schedule:
    print(f"{milestone['name']:35}: {milestone['date'].strftime('%d.%m.%Y')} ({milestone['work_day']} раб.день)")

# Расчет общей календарной длительности
project_start = milestone_schedule[0]['date']
project_finish = milestone_schedule[-1]['date']
total_calendar_days = (project_finish - project_start).days

print(f"\nОБЩАЯ ДЛИТЕЛЬНОСТЬ:")
print(f"- Рабочих дней: 66")
print(f"- Календарных дней: {total_calendar_days}")
print(f"- Плановое окончание: {project_finish.strftime('%d.%m.%Y')}")

print("\nДЕТАЛЬНЫЙ ГРАФИК ЗАДАЧ:")
print("-" * 60)
for task in calendar_schedule:
    if task['duration'] > 0:  # пропускаем фиктивные задачи
        print(f"Задача {task['task']:2}: {task['calendar_start'].strftime('%d.%m.%Y')} - {task['calendar_finish'].strftime('%d.%m.%Y')} "
              f"({task['duration']} дн.)")

In [ ]:
# Группировка задач по этапам проекта
project_stages = {
    'Этап 1: Аутентификация и профиль': [1, 2, 3, 4, 5],
    'Этап 2: Основные финансовые операции': [6, 7, 8, 9],
    'Этап 3: Система тегов и категорий': [10, 11, 12],
    'Этап 4: Уведомления и напоминания': [13, 14],
    'Этап 5: Планирование целей': [15, 16],
    'Этап 6: Аналитика и отчетность': [17, 18, 19, 20],
    'Этап 7: Дополнительные функции': [21, 22, 23, 24]
}

print("\nЭТАПЫ ПРОЕКТА С ДАТАМИ:")
print("=" * 50)

for stage_name, task_ids in project_stages.items():
    # Находим даты начала и окончания этапа
    stage_start_times = [start_times[task_id] for task_id in task_ids]
    stage_durations = [durations[task_id] for task_id in task_ids]
    stage_finish_times = [start_times[task_id] + durations[task_id] for task_id in task_ids]

    stage_start_day = min(stage_start_times)
    stage_finish_day = max(stage_finish_times)

    # Конвертируем в календарные даты
    start_calendar = calculate_calendar_schedule_simple([stage_start_day], [0], start_date)[0]['calendar_start']
    finish_calendar = calculate_calendar_schedule_simple([stage_finish_day], [0], start_date)[0]['calendar_start']

    print(f"{stage_name:35}: {start_calendar.strftime('%d.%m.%Y')} - {finish_calendar.strftime('%d.%m.%Y')}")
    print(f"{'':35}  ({stage_start_day}-{stage_finish_day} раб.дни)")